In [3]:
import sys, platform
import pvlib
import PySAM
import bifacial_radiance as br
import pandas as pd

In [4]:
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("Pvlib version: ", pvlib.__version__)
print("PySAM version: ", PySAM.__version__)
print("bifacial_radiance version: ", br.__version__)

Working on a  Windows 10
Python version  3.11.7 | packaged by Anaconda, Inc. | (main, Dec 15 2023, 18:05:47) [MSC v.1916 64 bit (AMD64)]
Pandas version  2.1.0
Pvlib version:  0.10.4.dev9+ge92ee94
PySAM version:  5.1.0
bifacial_radiance version:  0.4.2+340.g7909811


In [5]:
NREL_API_KEY = None  # <-- please set your NREL API key here
# note you must use "quotes" around your key as it is a string.

if NREL_API_KEY is None:
       NREL_API_KEY = 'DEMO_KEY'  # OK for this demo, but better to get your own key

In [6]:
import pvlib

metdata, metadata = pvlib.iotools.get_psm3(
    latitude=44.25, longitude=-117.33999633789062,
    api_key=NREL_API_KEY,
    email='silvana.ovaitt@nrel.gov',  # <-- any email works here fine
    names='tmy', map_variables=True)
metadata

{'Source': 'NSRDB',
 'Location ID': '200667',
 'City': '-',
 'State': '-',
 'Country': '-',
 'Time Zone': -7,
 'Local Time Zone': -7,
 'Dew Point Units': 'c',
 'DHI Units': 'w/m2',
 'DNI Units': 'w/m2',
 'GHI Units': 'w/m2',
 'Temperature Units': 'c',
 'Pressure Units': 'mbar',
 'Wind Direction Units': 'Degrees',
 'Wind Speed Units': 'm/s',
 'Surface Albedo Units': 'N/A',
 'Version': '3.2.0',
 'latitude': 44.25,
 'longitude': -117.34,
 'altitude': 979}

## 2. Modeling with bifacial_radiance

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import bifacial_radiance as br

In [8]:
br.__version__

'0.4.2+340.g7909811'

In [9]:
import os
from pathlib import Path

testfolder = 'TEMP'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in TEMP


In [11]:
radObj = br.RadianceObj('Sim3',path=testfolder, hpc=True)

path = TEMP
Making path: images
Making path: objects
Making path: results
Making path: skies
Making path: EPWs
Making path: materials


In [12]:
# Some of the names changed internally. While bifacial_radiance updates their expected names, we are renaming the values here
metadata['timezone'] = metadata['Time Zone']
metadata['county'] = '-'
metadata['elevation'] = metadata['altitude']
metadata['state'] = metadata['State']
metadata['country'] = metadata['Country']
metdata['Albedo'] = metdata['albedo']

Use NSRDBWeatherData to enter data the downloaded data in dataframe and dictionary forma for meteorological data and metadata respectively

In [19]:
#starttime can be 'MM_DD', or 'MM_DD_HH'
metData = radObj.readWeatherData(metadata, metdata, starttime='11_08_09', endtime='11_08_11',coerce_year=2021, label='center')

8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for center labeled data, at exact timestamp in input Weather File


In [ ]:
# -- establish tracking angles
hub_height = 1.5
pitch = 5
sazm = 180  # Tracker axis azimuth
modulename = 'PVmodule'
fixed_tilt_angle = None
gcr = 2 / pitch


trackerParams = {'limit_angle':50,
             'angledelta':5,
             'backtrack':True,
             'gcr':gcr,
             'cumulativesky':False,
             'azimuth': sazm,
             'fixed_tilt_angle': fixed_tilt_angle
             }

In [ ]:
trackerdict = radObj.set1axis(**trackerParams)

In [ ]:
radObj.setGround(0.2) 

In [ ]:
radObj.gendaylit1axis()

In [ ]:
module=radObj.makeModule(name=modulename, x=1,y=2)

In [ ]:
sceneDict = {'pitch':pitch, 
             'hub_height': hub_height,
             'nMods': 5,
             'nRows': 2,
             'tilt': fixed_tilt_angle,  
             'sazm': sazm
             }

In [ ]:
trackerdict = radObj.makeScene1axis(module=modulename,sceneDict=sceneDict)


In [ ]:
trackerdict = radObj.makeOct1axis()

In [ ]:
trackerdict = radObj.analysis1axis(customname = 'Module', 
                                   sensorsy=2, modWanted=2,
                                   rowWanted=1)

In [ ]:
trackerdict = radObj.calculateResults(bifacialityfactor=0.7, agriPV=False)

In [ ]:
radObj.CompiledResults

In [ ]:
resolutionGround = 1  #meter. use 1 for faster test runs
numsensors = int((pitch/resolutionGround)+1)
modscanback = {'xstart': 0, 
                'zstart': 0.05,
                'xinc': resolutionGround,
                'zinc': 0,
                'Ny':numsensors,
                'orient':'0 0 -1'}

# Analysis for GROUND
trackerdict = radObj.analysis1axis(customname = 'Ground', 
                                   modscanfront=modscanback, sensorsy=1)

In [ ]:
trackerdict = radObj.calculateResults(bifacialityfactor=0.7, agriPV=True)

In [ ]:
radObj.CompiledResults

##  Eploring Accessing the results directly since CompiledResults is failing for agriPV = False

## THIS WORKED WITH dev branch up to 4/22, and in the HPC versions we have.


In [ ]:
ResultPVWm2Back = list(radObj.CompiledResults['Grear_mean'])
ResultPVWm2Front = list(radObj.CompiledResults['Gfront_mean'])
ResultGHI = list(radObj.CompiledResults['GHI'])
ResultDHI = list(radObj.CompiledResults['DHI'])
ResultDNI = list(radObj.CompiledResults['DNI'])
ResultPout = list(radObj.CompiledResults['Pout'])
ResultWindSpeed = list(radObj.CompiledResults['Wind Speed'])
ResultPVWm2Back


In [ ]:
# In another wranch?? Thsi hsould have worked
#list(radObj.CompiledResults['Module_temp'])


In [ ]:
keys=list(trackerdict.keys())

groundIrrad = []
temp_air = []
pitch= []
for key in keys:
    groundIrrad.append(trackerdict[key]['Results'][1]['Wm2Front'])
    temp_air.append(trackerdict[key]['temp_air'])
    pitch.append(trackerdict[key]['scene'].sceneDict['pitch'])
    

In [ ]:
results = pd.DataFrame(list(zip(ResultPVWm2Back, ResultPVWm2Front)), columns = ["Back","Front"])

In [ ]:
results['pitch']=trackerdict[key]['scene'].sceneDict['pitch']

In [ ]:
results.to_pickle(results_path)